In [34]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스)
# ----------------------
N = 3        # 선박 수
T = 10       # 전체 시간 슬롯 (1시간 단위: 1,...,10)
Q = 2        # 크레인 수

# 선박별 총 작업시간
a = {1: 4, 2: 3, 3: 5}

# 입항 및 출항 시각 (t 단위)
b = {1: 1, 2: 3, 3: 2}
d = {1: 8, 2: 10, 3: 9}

# 시간당 최소/최대 크레인 할당 (여기서는 0~1로 제한)
qmin = {1: 0, 2: 0, 3: 0}
qmax = {1: 1, 2: 1, 3: 1}

# 계약시간 (최대 계약시간)
H = 3

# 단가
C3 = 10    # 계약시간 단가
C4 = 20    # 초과근무 단가

# 야간 할증 계수: t=5,6는 야간(1.5), 나머지는 주간(1.0)
w = {t: 1.5 if t in [5,6] else 1.0 for t in range(1, T+1)}

# ----------------------
# 2. 모델 생성
# ----------------------
model = Model("TimeVariant_QCAP")

# ----------------------
# 3. 결정변수
# ----------------------
# x^c[i,t,m] : 선박 i가 시간 t에 크레인 m을 계약시간으로 할당하면 1, 아니면 0
# x^o[i,t,m] : 선박 i가 시간 t에 크레인 m을 초과근무로 할당하면 1, 아니면 0

x_c = {}
x_o = {}

for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            x_c[i, t, m] = model.addVar(vtype=GRB.BINARY, name=f"x_c_{i}_{t}_{m}")
            x_o[i, t, m] = model.addVar(vtype=GRB.BINARY, name=f"x_o_{i}_{t}_{m}")

model.update()

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(1, b[i]):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_o")
        for t in range(d[i], T+1):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_o")

# 4.2. 총 작업량 충족
# 각 선박은 입항 시각부터 출항 직전까지 총 작업시간이 a[i]가 되어야 함.
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] + x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1)) == a[i],
        name=f"total_work_{i}"
    )

# 4.3. 시간 슬롯별 최소/최대 크레인 할당 (선박별)
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1)) >= qmin[i],
            name=f"min_qc_{i}_{t}"
        )
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1)) <= qmax[i],
            name=f"max_qc_{i}_{t}"
        )

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당 (크레인별)
for t in range(1, T+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for i in range(1, N+1)) <= 1,
            name=f"one_vessel_per_crane_{t}_{m}"
        )

# 4.5. 계약시간/초과근무 분리
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1)) == min(H, a[i]),
        name=f"contract_time_{i}"
    )
    model.addConstr(
        quicksum(x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1)) == a[i] - min(H, a[i]),
        name=f"overtime_{i}"
    )

# ----------------------
# 5. 목적함수
# ----------------------
model.setObjective(
    quicksum(
        w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m])
        for i in range(1, N+1)
        for t in range(1, T+1)
        for m in range(1, Q+1)
    ),
    GRB.MINIMIZE
)

model.update()

# ----------------------
# 6. 최적화 및 결과 출력
# ----------------------
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for t in range(b[i], d[i]):
            for m in range(1, Q+1):
                val_c = x_c[i,t,m].X
                val_o = x_o[i,t,m].X
                if abs(val_c) > 1e-6 or abs(val_o) > 1e-6:
                    print(f"  t={t}, m={m}: x^c = {val_c}, x^o = {val_o}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 107 rows, 120 columns and 492 nonzeros
Model fingerprint: 0x9ea3ffeb
Variable types: 0 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 175.0000000
Presolve removed 68 rows and 53 columns
Presolve time: 0.00s
Presolved: 39 rows, 67 columns, 197 nonzeros
Variable types: 0 continuous, 67 integer (67 binary)
Found heuristic solution: objective 155.0000000

Root relaxation: objective 1.500000e+02, 26 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0 

In [24]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스 확장)
# ----------------------
N = 5        # 선박 수
T = 20       # 시간 슬롯 (1시간 단위: t=1,...,10)
Q = 10        # 크레인 수

# 선박별 총 작업시간 (크레인-시간)
a = {1: 4, 2: 3, 3: 5, 4: 4, 5: 6}

# 입항 및 출항 시각 (시간 단위)
b = {1: 10, 2: 15, 3: 5, 4: 1, 5: 2}
d = {1: 18, 2: 20, 3: 11, 4: 5, 5: 9}

# 시간당 최소/최대 크레인 할당 (여기서는 0~1)
qmin = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
qmax = {1: 2, 2: 3, 3: 3, 4: 2, 5: 1}

# 계약시간 상한 (최대 계약시간)
H = 10

# Big-M 값들
M = 999     # x^c, x^o 관련 Big-M (여기서는 999로 둠)
M1 = 10    # (57) 제약용 Big-M (여기서는 100으로, 필요시 조정)

# 단가 및 야간 할증 계수
C3 = 10    # 계약시간 단가
C4 = 20    # 초과근무 단가
# 예시: 시간 t=5,6는 야간(할증 계수 1.5), 나머지는 주간(1.0)
w = {t: 1.5 if t in [5,6] else 1.0 for t in range(1, T+1)}


# ----------------------
# 2. 모델 생성
# ----------------------
model = Model("TimeVariant_QCAP_with_Contiguity")

# ----------------------
# 3. 결정변수
# ----------------------
# x^c[i,t,m]: 선박 i가 시간 t에 크레인 m을 계약시간으로 할당 (0/1)
# x^o[i,t,m]: 선박 i가 시간 t에 크레인 m을 초과근무로 할당 (0/1)
# r[i,t,m]: 시간 t에 선박 i가 크레인 m을 할당했는지 (r = x^c + x^o)
# s[i,t,m]: 시간 t에 작업을 새로 시작 (0→1 전이)하면 1
# e[i,t,m]: 시간 t에 작업을 종료 (1→0 전이)하면 1

x_c = {}
x_o = {}
r_var = {}
s_var = {}
e_var = {}
z = {}
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            x_c[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_c_{i}_{t}_{m}")
            x_o[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_o_{i}_{t}_{m}")
            # 할당 여부 변수: r = x_c + x_o
            r_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"r_{i}_{t}_{m}")
            s_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"s_{i}_{t}_{m}")
            e_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"e_{i}_{t}_{m}")
            z[i,t,m] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"z_{i}_{t}_{m}")

model.update()

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(1, b[i]):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_o")
        for t in range(d[i], T+1):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_o")

# 4.2. 총 작업량 충족
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] + x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == a[i],
        name=f"total_work_{i}"
    )

# 4.3. 시간 슬롯별 최소/최대 크레인 (선박별)
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1)) >= qmin[i],
            name=f"min_qc_{i}_{t}"
        )
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1)) <= qmax[i],
            name=f"max_qc_{i}_{t}"
        )

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당 (크레인별)
for t in range(1, T+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for i in range(1, N+1)) <= 1,
            name=f"one_vessel_per_crane_{t}_{m}"
        )



# 4.5. 계약시간/초과 근무 분리
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == min(H, a[i]),
        name=f"contract_time_{i}"
    )
    model.addConstr(
        quicksum(x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == a[i] - min(H, a[i]),
        name=f"overtime_{i}"
    )

# 4.6. r 변수와의 연결: r[i,t,m] = x_c[i,t,m] + x_o[i,t,m]
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            model.addConstr(
                r_var[i,t,m] == x_c[i,t,m] + x_o[i,t,m],
                name=f"r_link_{i}_{t}_{m}"
            )

# 4.7. 연속 블록 제약 (시작/종료 이벤트)
# (A) 전이 제약: t = b[i]+1 ... d[i]-1
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                r_var[i,t,m] - r_var[i,t-1,m] == s_var[i,t,m] - e_var[i,t,m],
                name=f"cont_{i}_{t}_{m}"
            )
# r_var[i,t,m] : 선박 i, 시간 t, 크레인 m 할당 여부 (0/1)
# 크레인 연속성 제약:
for i in range(1, N+1):
    for t in range(1, T+1):
        # m < n
        for m_idx in range(1, Q):
            for n_idx in range(m_idx+1, Q+1):
                # 중간 크레인 k
                for k in range(m_idx+1, n_idx):
                    model.addConstr(
                        r_var[i,t,m_idx] + r_var[i,t,n_idx] - 1 <= r_var[i,t,k],
                        name=f"contiguous_{i}_{t}_{m_idx}_{n_idx}_{k}"
                    )


# (B) 초기 조건: t = b[i]
for i in range(1, N+1):
    for m in range(1, Q+1):
        t0 = b[i]
        model.addConstr(s_var[i,t0,m] == r_var[i,t0,m], name=f"s_init_{i}_{t0}_{m}")
        model.addConstr(e_var[i,t0,m] == 0, name=f"e_init_{i}_{t0}_{m}")

# (C) 단일 시작/종료: 각 선박·크레인에 대해
for i in range(1, N+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(s_var[i,t,m] for t in range(b[i], d[i])) <= 1,
            name=f"one_start_{i}_{m}"
        )
        model.addConstr(
            quicksum(e_var[i,t,m] for t in range(b[i], d[i])) <= 1,
            name=f"one_end_{i}_{m}"
        )

# (D) 이벤트 발생 보완: t = b[i]+1 ... d[i]-1
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                s_var[i,t,m] <= 1 - r_var[i,t-1,m],
                name=f"s_bound_{i}_{t}_{m}"
            )
            model.addConstr(
                e_var[i,t,m] <= r_var[i,t-1,m],
                name=f"e_bound_{i}_{t}_{m}"
            )

# 4.8. (57) 추가 제약: z 제한 – 시간 t에서 시작한 작업은, 
# 종료 이벤트가 발생한 t'까지의 간격을 초과할 수 없다.
# t = b[i] ... d[i]-1, t' = t+1 ... d[i]
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            for tp in range(t+1, d[i]+1):
                model.addConstr(
                    z[i,t,m] <= (tp - t) + M1*(1 - e_var[i,tp,m]),
                    name=f"z_limit_{i}_{t}_{tp}_{m}"
                )

# 4.9. z 관련 제약: 
# (i) z <= M * s  (이미 있음) 및 z <= r (시간 슬롯 1시간이므로)
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            model.addConstr(z[i,t,m] <= M * s_var[i,t,m], name=f"z_le_Ms_{i}_{t}_{m}")
            model.addConstr(z[i,t,m] <= r_var[i,t,m], name=f"z_bound_{i}_{t}_{m}")
# (ii) z <= H
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            model.addConstr(z[i,t,m] <= H, name=f"z_le_H_{i}_{t}_{m}")

model.update()

# ----------------------
# 5. 목적함수
# ----------------------
# 최종 비용: 각 시간 슬롯에서 할증 계수 w_t를 곱해 계약시간, 초과근무 단가 적용
model.setObjective(
    quicksum(
        w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m])
        for i in range(1, N+1)
        for t in range(1, T+1)
        for m in range(1, Q+1)
    ),
    GRB.MINIMIZE
)

model.update()

# ----------------------
# 6. 최적화 및 결과 출력
# ----------------------
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for t in range(b[i], d[i]):
            for m in range(1, Q+1):
                val_c = x_c[i,t,m].X
                val_o = x_o[i,t,m].X
                val_s = s_var[i,t,m].X
                val_e = e_var[i,t,m].X
                if abs(val_c) > 1e-6 or abs(val_o) > 1e-6 or abs(val_s) > 1e-6 or abs(val_e) > 1e-6:
                    print(f"  t={t}, m={m}: x^c={val_c}, x^o={val_o}, s={val_s}, e={val_e}")
else:
    print(f"No optimal solution found. (Status: {model.status})")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17625 rows, 6000 columns and 51250 nonzeros
Model fingerprint: 0x93bbb6a0
Variable types: 1000 continuous, 5000 integer (5000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 13925 rows and 5200 columns
Presolve time: 0.02s
Presolved: 3700 rows, 800 columns, 11500 nonzeros
Variable types: 0 continuous, 800 integer (800 binary)
Found heuristic solution: objective 235.0000000

Root relaxation: objective 2.250000e+02, 74 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     225.00

In [25]:
# 결과 출력: 선박별, 크레인별로 출력
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for m in range(1, Q+1):
            print(f"  크레인 {m}:")
            for t in range(b[i], d[i]):
                val_c = x_c[i,t,m].X
                val_o = x_o[i,t,m].X
                val_s = s_var[i,t,m].X
                val_e = e_var[i,t,m].X
                # 값이 있는 경우에만 출력
                if abs(val_c) > 1e-6 or abs(val_o) > 1e-6 or abs(val_s) > 1e-6 or abs(val_e) > 1e-6:
                    print(f"    t={t}: x^c={val_c}, x^o={val_o}, s={val_s}, e={val_e}")
else:
    print(f"No optimal solution found. (Status: {model.status})")


Optimal solution found:
Objective value = 225.0

선박 1:
  크레인 1:
    t=17: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 2:
    t=17: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 3:
  크레인 4:
  크레인 5:
  크레인 6:
  크레인 7:
  크레인 8:
  크레인 9:
    t=14: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
    t=15: x^c=0.0, x^o=0.0, s=-0.0, e=1.0
  크레인 10:
    t=14: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
    t=15: x^c=0.0, x^o=0.0, s=-0.0, e=1.0

선박 2:
  크레인 1:
    t=19: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 2:
    t=19: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 3:
    t=19: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 4:
  크레인 5:
  크레인 6:
  크레인 7:
  크레인 8:
  크레인 9:
  크레인 10:

선박 3:
  크레인 1:
    t=9: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
    t=10: x^c=0.0, x^o=0.0, s=-0.0, e=1.0
  크레인 2:
    t=9: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
    t=10: x^c=0.0, x^o=0.0, s=-0.0, e=1.0
  크레인 3:
  크레인 4:
  크레인 5:
  크레인 6:
  크레인 7:
    t=10: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 8:
    t=10: x^c=1.0, x^o=0.0, s=1.0, e=-0.0
  크레인 9:
    t=10: x^c=1.0, x^o=0.0, s=1.0, e=-0

In [86]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스)
# ----------------------
N = 4        # 선박 수
T = 24       # 전체 시간 슬롯 (1시간 단위: 1,...,10)
Q = 4        # 크레인 수

# 선박별 총 작업시간
a = {1: 5, 2: 5, 3: 5, 4: 4}

# 입항 및 출항 시각 (t 단위)
b = {1: 1, 2: 3, 3: 2, 4:11}
d = {1: 8, 2: 10, 3: 9, 4:17}

# 시간당 최소/최대 크레인 할당 (여기서는 0~1로 제한)
qmin = {1: 0, 2: 0, 3: 0, 4:0}
qmax = {1: 3, 2: 3, 3: 3, 4:3}

# 계약시간 (최대 계약시간)
H = 3

# 단가
C3 = 10    # 계약시간 단가
C4 = 20    # 초과근무 단가

# 야간 할증 계수: t=5,6는 야간(1.5), 나머지는 주간(1.0)
w = {t: 1.5 if t in [18,19,20,21,22,23,24,0,1,2,3,4,5,6] else 1.0 for t in range(1, T+1)}


# Big-M 값들
M = 999     
M1 = 10   # (57) 제약용 Big-M
# ----------------------
# 2. 모델 생성
# ----------------------
model = Model("TimeVariant_QCAP_with_Contiguity")

# ----------------------
# 3. 결정변수
# ----------------------
# x^c[i,t,m]: 선박 i가 시간 t에 크레인 m을 계약시간으로 할당 (0/1)
# x^o[i,t,m]: 선박 i가 시간 t에 크레인 m을 초과근무로 할당 (0/1)
# r[i,t,m]: 할당 여부, r = x^c + x^o
# s[i,t,m]: 시간 t에 작업 시작 이벤트 (0→1 전이) (0/1)
# e[i,t,m]: 시간 t에 작업 종료 이벤트 (1→0 전이) (0/1)
# z[i,t,m]: (추가) 작업 시작 시점에서 종료 이벤트까지의 시간 간격 제한에 사용 (연속 변수)

x_c = {}
x_o = {}
r_var = {}
s_var = {}
e_var = {}
z = {}

for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            x_c[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_c_{i}_{t}_{m}")
            x_o[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_o_{i}_{t}_{m}")
            r_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"r_{i}_{t}_{m}")
            s_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"s_{i}_{t}_{m}")
            e_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"e_{i}_{t}_{m}")
            z[i,t,m]   = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"z_{i}_{t}_{m}")

model.update()

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(1, b[i]):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_o")
        for t in range(d[i], T+1):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_o")

# 4.2. 총 작업량 충족 (입항 시각부터 출항 직전까지)
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] + x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == a[i],
        name=f"total_work_{i}"
    )

# 4.3. 시간 슬롯별 최소/최대 크레인 (선박별)
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1))
            >= qmin[i],
            name=f"min_qc_{i}_{t}"
        )
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1))
            <= qmax[i],
            name=f"max_qc_{i}_{t}"
        )

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당 (크레인별)
for t in range(1, T+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for i in range(1, N+1))
            <= 1,
            name=f"one_vessel_per_crane_{t}_{m}"
        )

# 4.5. 계약시간/초과 근무 분리
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == min(H, a[i]),
        name=f"contract_time_{i}"
    )
    model.addConstr(
        quicksum(x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == a[i] - min(H, a[i]),
        name=f"overtime_{i}"
    )

# 4.6. r 변수와의 연결: r[i,t,m] = x^c[i,t,m] + x^o[i,t,m]
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            model.addConstr(
                r_var[i,t,m] == x_c[i,t,m] + x_o[i,t,m],
                name=f"r_link_{i}_{t}_{m}"
            )

# 4.7. 연속 블록 제약 (시작/종료 이벤트)
# (A) 시간 전이 제약: t = b[i]+1, ..., d[i]-1
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                r_var[i,t,m] - r_var[i,t-1,m] == s_var[i,t,m] - e_var[i,t,m],
                name=f"cont_{i}_{t}_{m}"
            )
# (B) 초기 조건: t = b[i]
for i in range(1, N+1):
    for m in range(1, Q+1):
        t0 = b[i]
        model.addConstr(s_var[i,t0,m] == r_var[i,t0,m], name=f"s_init_{i}_{t0}_{m}")
        model.addConstr(e_var[i,t0,m] == 0, name=f"e_init_{i}_{t0}_{m}")
# (C) 단일 시작/종료: 각 선박·크레인에 대해 (전체 작업기간 동안 시작과 종료는 각각 한 번 발생)
for i in range(1, N+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(s_var[i,t,m] for t in range(b[i], d[i]))
            == 1,
            name=f"one_start_{i}_{m}"
        )
        model.addConstr(
            quicksum(e_var[i,t,m] for t in range(b[i], d[i]))
            == 1,
            name=f"one_end_{i}_{m}"
        )
# (D) 이벤트 발생 보완: t = b[i]+1, ..., d[i]-1
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                s_var[i,t,m] <= 1 - r_var[i,t-1,m],
                name=f"s_bound_{i}_{t}_{m}"
            )
            model.addConstr(
                e_var[i,t,m] <= r_var[i,t-1,m],
                name=f"e_bound_{i}_{t}_{m}"
            )

# 4.8. 크레인 연속성 제약 (같은 시간 슬롯에서 할당된 크레인 인덱스는 연속되어야 함)
# 만약 선박 i가 시간 t에 크레인 m와 n (m<n)을 할당받았다면, 그 사이의 모든 크레인 k (m < k < n)도 할당되어야 함.
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        for m_idx in range(1, Q):
            for n_idx in range(m_idx+1, Q+1):
                for k in range(m_idx+1, n_idx):
                    model.addConstr(
                        r_var[i,t,m_idx] + r_var[i,t,n_idx] - 1 <= r_var[i,t,k],
                        name=f"contiguous_{i}_{t}_{m_idx}_{n_idx}_{k}"
                    )

# 4.9. (57) 추가 제약: z 제한 – t에서 시작한 작업은, 종료 이벤트가 발생한 t'까지의 간격을 초과할 수 없음.
# t = b[i] ... d[i]-1, t' = t+1 ... d[i]
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            for tp in range(t+1, d[i]+1):
                model.addConstr(
                    z[i,t,m] <= (tp - t) + M1*(1 - e_var[i,tp,m]),
                    name=f"z_limit_{i}_{t}_{tp}_{m}"
                )

# 4.10. z 관련 제약:
# (i) z <= M * s (이미 있으므로, 여기서는 추가로 z가 1시간 슬롯을 넘지 못하도록)
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            model.addConstr(z[i,t,m] <= M * s_var[i,t,m], name=f"z_le_Ms_{i}_{t}_{m}")
            model.addConstr(z[i,t,m] <= r_var[i,t,m], name=f"z_bound_{i}_{t}_{m}")
# (ii) z <= H
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            model.addConstr(z[i,t,m] <= H, name=f"z_le_H_{i}_{t}_{m}")

model.update()

# ----------------------
# 5. 목적함수
# ----------------------
model.setObjective(
    quicksum(
        w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m])
        for i in range(1, N+1)
        for t in range(1, T+1)
        for m in range(1, Q+1)
    ),
    GRB.MINIMIZE
)

model.update()

# ----------------------
# 6. 최적화 및 결과 출력 (크레인 별로 그룹화)
# ----------------------
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}\n")
    
    print("각 선박별, 크레인별 변수 값 (모든 t):")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for m in range(1, Q+1):
            print(f"  크레인 {m}:")
            for t in range(1, T+1):
                # 모든 t에 대해 출력 (할당 여부, 시작, 종료 이벤트 등)
                print(f"    t={t}: r = {r_var[i,t,m].X:.0f}, x^c = {x_c[i,t,m].X:.0f}, x^o = {x_o[i,t,m].X:.0f}, s = {s_var[i,t,m].X:.0f}, e = {e_var[i,t,m].X:.0f}")
else:
    print(f"No optimal solution found. (Status: {model.status})")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2290 rows, 2304 columns and 6040 nonzeros
Model fingerprint: 0xfc6b7005
Variable types: 384 continuous, 1920 integer (1920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1906 rows and 1968 columns
Presolve time: 0.01s
Presolved: 384 rows, 336 columns, 1448 nonzeros
Variable types: 0 continuous, 336 integer (336 binary)

Root relaxation: objective 3.100000e+02, 267 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     310.0000000  310.00000  0.00%     -    0s

Explored 1 nodes

In [81]:
# 최적화
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}\n")
    
    # 할증 계수 출력
    print("시간 슬롯별 할증 계수:")
    for t in range(1, T+1):
        print(f"  t={t}: w = {w[t]}")
    
    print("\n각 선박별, 크레인별 변수 값 (모든 t):")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for m in range(1, Q+1):
            print(f"  크레인 {m}:")
            for t in range(1, T+1):
                print(f"    t={t}: r = {r_var[i,t,m].X:.0f}, x^c = {x_c[i,t,m].X:.0f}, x^o = {x_o[i,t,m].X:.0f}, s = {s_var[i,t,m].X:.0f}, e = {e_var[i,t,m].X:.0f}")
else:
    print(f"No optimal solution found. (Status: {model.status})")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1680 rows, 1728 columns and 4368 nonzeros
Model fingerprint: 0x23d5fc63
Variable types: 288 continuous, 1440 integer (1440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolved: 231 rows, 243 columns, 870 nonzeros

Continuing optimization...


Explored 1 nodes (285 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 315 320 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.150000000000e+02, best bound 3.150000000000e+02, gap 0.0000%

Optimal solution found:
Objective value = 315.0

시간 슬롯별 할증 계수:
  t=1: w = 1.5
  t=2: w = 1.5
  t=3: w = 1.5
  t=4: w = 1.5
  t=5: w = 1.5
  t=6: w = 

In [70]:
model.computeIIS()
model.write("model.ilp")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0     11457         -         0      1200         -           0s
       26        26        26         0         0         0           0s

IIS computed: 26 constraints, 0 bounds
IIS runtime: 0.39 seconds (0.14 work units)


In [82]:
from gurobipy import Model, GRB, quicksum

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스)
# ----------------------
N = 4        # 선박 수
T = 24       # 전체 시간 슬롯 (1시간 단위: 1,...,24)
Q = 3        # 크레인 수

# 선박별 총 작업시간
a = {1: 5, 2: 5, 3: 5, 4: 4}

# 입항 및 출항 시각 (t 단위)
b = {1: 1, 2: 3, 3: 2, 4:11}
d = {1: 8, 2: 10, 3: 9, 4:17}

# 시간당 최소/최대 크레인 할당 (각 선박별)
qmin = {1: 0, 2: 0, 3: 0, 4: 0}
qmax = {1: 3, 2: 3, 3: 3, 4: 3}

# 계약시간 (최대 계약시간)
H = 3

# 단가
C3 = 10    # 계약시간 단가
C4 = 20    # 초과근무 단가

# 야간 할증 계수: 예시로 t=5,6는 야간(1.5), 나머지는 주간(1.0)
w = {t: 1.5 if t in [18,19,20,21,22,23,24,0,1,2,3,4,5,6] else 1.0 for t in range(1, T+1)}

# Big-M 값들
M = 999     
M1 = 10   # (57) 제약용 Big-M

# ----------------------
# 2. 모델 생성
# ----------------------
model = Model("TimeVariant_QCAP_with_Contiguity")

# ----------------------
# 3. 결정변수
# ----------------------
# x^c[i,t,m]: 선박 i가 시간 t에 크레인 m을 계약시간으로 할당 (0/1)
# x^o[i,t,m]: 선박 i가 시간 t에 크레인 m을 초과근무로 할당 (0/1)
# r[i,t,m]: 할당 여부, r = x^c + x^o
# s[i,t,m]: 시간 t에 작업 시작 이벤트 (0→1 전이) (0/1)
# e[i,t,m]: 시간 t에 작업 종료 이벤트 (1→0 전이) (0/1)
# z[i,t,m]: 작업 시작 시점부터 종료 이벤트까지의 시간 간격 제한 (연속 변수)
x_c = {}
x_o = {}
r_var = {}
s_var = {}
e_var = {}
z = {}

for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            x_c[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_c_{i}_{t}_{m}")
            x_o[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"x_o_{i}_{t}_{m}")
            r_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"r_{i}_{t}_{m}")
            s_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"s_{i}_{t}_{m}")
            e_var[i,t,m] = model.addVar(vtype=GRB.BINARY, name=f"e_{i}_{t}_{m}")
            z[i,t,m]   = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"z_{i}_{t}_{m}")

model.update()

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(1, b[i]):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_before_{i}_{t}_{m}_o")
        for t in range(d[i], T+1):
            model.addConstr(x_c[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_c")
            model.addConstr(x_o[i,t,m] == 0, name=f"no_assign_after_{i}_{t}_{m}_o")

# 4.2. 총 작업량 충족 (입항 시각부터 출항 직전까지)
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] + x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == a[i],
        name=f"total_work_{i}"
    )

# 4.3. 시간 슬롯별 최소/최대 크레인 (선박별)
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1))
            >= qmin[i],
            name=f"min_qc_{i}_{t}"
        )
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for m in range(1, Q+1))
            <= qmax[i],
            name=f"max_qc_{i}_{t}"
        )

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당 (크레인별)
for t in range(1, T+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(x_c[i,t,m] + x_o[i,t,m] for i in range(1, N+1))
            <= 1,
            name=f"one_vessel_per_crane_{t}_{m}"
        )

# 4.5. 계약시간/초과 근무 분리
for i in range(1, N+1):
    model.addConstr(
        quicksum(x_c[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == min(H, a[i]),
        name=f"contract_time_{i}"
    )
    model.addConstr(
        quicksum(x_o[i,t,m] for t in range(b[i], d[i]) for m in range(1, Q+1))
        == a[i] - min(H, a[i]),
        name=f"overtime_{i}"
    )

# 4.6. r 변수와의 연결: r[i,t,m] = x_c[i,t,m] + x_o[i,t,m]
for i in range(1, N+1):
    for t in range(1, T+1):
        for m in range(1, Q+1):
            model.addConstr(
                r_var[i,t,m] == x_c[i,t,m] + x_o[i,t,m],
                name=f"r_link_{i}_{t}_{m}"
            )

# 4.7. 연속 블록 제약 (시작/종료 이벤트)
# (A) 시간 전이 제약: t = b[i]+1, ..., d[i]-1
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i]+1, d[i]):
            model.addConstr(
                r_var[i,t,m] - r_var[i,t-1,m] == s_var[i,t,m] - e_var[i,t,m],
                name=f"cont_{i}_{t}_{m}"
            )
# (B) 초기 조건: t = b[i]
for i in range(1, N+1):
    for m in range(1, Q+1):
        t0 = b[i]
        model.addConstr(s_var[i,t0,m] == r_var[i,t0,m], name=f"s_init_{i}_{t0}_{m}")
        model.addConstr(e_var[i,t0,m] == 0, name=f"e_init_{i}_{t0}_{m}")
# (C) 단일 시작/종료: 각 선박·크레인에 대해, 전체 작업기간 동안 시작과 종료는 각각 한 번 발생
for i in range(1, N+1):
    for m in range(1, Q+1):
        model.addConstr(
            quicksum(s_var[i,t,m] for t in range(b[i], d[i]))
            == 1,
            name=f"one_start_{i}_{m}"
        )
        model.addConstr(
            quicksum(e_var[i,t,m] for t in range(b[i], d[i]))
            == 1,
            name=f"one_end_{i}_{m}"
        )
# (D) **[제거됨]**: 이벤트 발생 보완 제약 (s_bound, e_bound)를 제거

# 4.8. 크레인 연속성 제약 (같은 시간 슬롯에서 할당된 크레인 인덱스는 연속되어야 함)
# 만약 선박 i가 시간 t에 크레인 m와 n (m<n)을 할당받았다면, 그 사이의 모든 크레인 k (m < k < n)도 할당되어야 함.
for i in range(1, N+1):
    for t in range(b[i], d[i]):
        for m_idx in range(1, Q):
            for n_idx in range(m_idx+1, Q+1):
                for k in range(m_idx+1, n_idx):
                    model.addConstr(
                        r_var[i,t,m_idx] + r_var[i,t,n_idx] - 1 <= r_var[i,t,k],
                        name=f"contiguous_{i}_{t}_{m_idx}_{n_idx}_{k}"
                    )

# 4.9. 추가 제약: z 제한 – t에서 시작한 작업은, 종료 이벤트가 발생한 t'까지의 간격을 초과할 수 없음.
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            for tp in range(t+1, d[i]+1):
                model.addConstr(
                    z[i,t,m] <= (tp - t) + M1*(1 - e_var[i,tp,m]),
                    name=f"z_limit_{i}_{t}_{tp}_{m}"
                )

# 4.10. z 관련 제약:
# (i) z <= M * s
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            model.addConstr(z[i,t,m] <= M * s_var[i,t,m], name=f"z_le_Ms_{i}_{t}_{m}")
            # model.addConstr(z[i,t,m] <= r_var[i,t,m], name=f"z_bound_{i}_{t}_{m}")  # 제거함
# (ii) z <= H
for i in range(1, N+1):
    for m in range(1, Q+1):
        for t in range(b[i], d[i]):
            model.addConstr(z[i,t,m] <= H, name=f"z_le_H_{i}_{t}_{m}")

model.update()

# ----------------------
# 5. 목적함수
# ----------------------
model.setObjective(
    quicksum(
        w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m])
        for i in range(1, N+1)
        for t in range(1, T+1)
        for m in range(1, Q+1)
    ),
    GRB.MINIMIZE
)

model.update()

# ----------------------
# 6. 최적화 및 결과 출력 (크레인 별로 그룹화)
# ----------------------
model.optimize()

if model.status == GRB.OPTIMAL:
    print("\nOptimal solution found:")
    print(f"Objective value = {model.ObjVal}\n")
    
    print("시간 슬롯별 할증 계수:")
    for t in range(1, T+1):
        print(f"  t={t}: w = {w[t]}")
    
    print("\n각 선박별, 크레인별 변수 값 (모든 t):")
    for i in range(1, N+1):
        print(f"\n선박 {i}:")
        for m in range(1, Q+1):
            print(f"  크레인 {m}:")
            for t in range(1, T+1):
                print(f"    t={t}: r = {r_var[i,t,m].X:.0f}, x^c = {x_c[i,t,m].X:.0f}, x^o = {x_o[i,t,m].X:.0f}, s = {s_var[i,t,m].X:.0f}, e = {e_var[i,t,m].X:.0f}")
else:
    print(f"No optimal solution found. (Status: {model.status})")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1461 rows, 1728 columns and 3930 nonzeros
Model fingerprint: 0x44f6720f
Variable types: 288 continuous, 1440 integer (1440 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+01, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 1320 rows and 1485 columns
Presolve time: 0.00s
Presolved: 141 rows, 243 columns, 726 nonzeros
Variable types: 0 continuous, 243 integer (243 binary)

Root relaxation: objective 3.150000e+02, 130 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                     315.0000000  155.00000  50.8%     -    0s
     0     0      